In [148]:
from openai import OpenAI
import os
import pandas as pd
from pydantic import BaseModel, Field
from typing import List
from tqdm import tqdm
import json
from random import sample
import jsonlines as jsonl
from openai import OpenAI



In [136]:
class ArgumentClassification(BaseModel):
    label: int = Field(description="The label associated with the argument (0 or 1)")

In [110]:
# Dictionary for label-to-argument mappings for each topic
topic_label_to_argument = {
    "abortion": {
        "p-right": "Abortion is a woman’s right.",
        "p-rape": "Rape victims need it to be legal.",
        "p-not_human": "A fetus is not a human yet, so it's okay to abort.",
        "p-mother_danger": "Abortion should be allowed when a mother's life is in danger.",
        "p-baby_ill_treatment": "Unwanted babies are ill-treated by parents and/or not always adopted.",
        "p-birth_ctrl": "Birth control fails at times and abortion is one way to deal with it.",
        "p-not_murder": "Abortion is not murder.",
        "p-sick_mom": "Mother is not healthy/financially solvent.",
        "p-other": "Others",
        "c-adopt": "Put baby up for adoption.",
        "c-kill": "Abortion kills a life.",
        "c-baby_right": "An unborn baby is a human and has the right to live.",
        "c-sex": "Be willing to have the baby if you have sex.",
        "c-bad_4_mom": "Abortion is harmful for women.",
        "c-other": "Others"
    },
    "gayRights": {
        "p-normal": "Gay marriage is like any other marriage.",
        "p-right_denied": "Gay people should have the same rights as straight people.",
        "p-no_threat_for_child": "Gay parents can adopt and ensure a happy life for a baby.",
        "p-born": "People are born gay.",
        "p-religion": "Religion should not be used against gay rights.",
        "p-Other": "Others",
        "c-religion": "Religion does not permit gay marriages.",
        "c-abnormal": "Gay marriages are not normal/against nature.",
        "c-threat_to_child": "Gay parents cannot raise kids properly.",
        "c-gay_problems": "Gay people have problems and create social issues.",
        "c-Other": "Others"
    },
    "obama": {
        "p-economy": "Fixed the economy.",
        "p-War": "Ending the wars.",
        "p-republicans": "Better than the republican candidates.",
        "p-decision_policies": "Makes good decisions/policies.",
        "p-quality": "Has qualities of a good leader.",
        "p-health": "Ensured better healthcare.",
        "p-foreign_policies": "Executed effective foreign policies.",
        "p-job": "Created more jobs.",
        "p-Other": "Others",
        "c-economy": "Destroyed our economy.",
        "c-War": "Wars are still on.",
        "c-job": "Unemployment rate is high.",
        "c-health": "Healthcare bill is a failure.",
        "c-decision_policies": "Poor decision-maker.",
        "c-republicans": "We have better republicans than Obama.",
        "c-quality": "Not eligible as a leader.",
        "c-foreign_policies": "Ineffective foreign policies.",
        "c-Other": "Others"
    },
    "marijuana": {
        "p-not_addictive": "Not addictive.",
        "p-medicine": "Used as a medicine for its positive effects.",
        "p-legal": "Legalized marijuana can be controlled and regulated by the government.",
        "p-right": "Prohibition violates human rights.",
        "p-no_damage": "Does not cause any damage to our bodies.",
        "p-Other": "Others",
        "c-health": "Damages our bodies.",
        "c-mind": "Responsible for brain damage.",
        "c-illegal": "If legalized, people will use marijuana and other drugs more.",
        "c-crime": "Causes crime.",
        "c-addiction": "Highly addictive.",
        "c-Other": "Others"
    }
}

In [116]:
def prep_fewshot_samples(samples_file, topic):
    df = pd.read_csv(samples_file)
    output = ''
    for i, row in df.iterrows():
        comment = row['text']
        output = f"{output}\n Comment: {comment}\n The following arguments are present (1) or not present (0) in this comment:\n"
        argument_type = row['label']
        argument = topic_label_to_argument[topic][argument_type]
        output = f"{output} Argument {i}: {argument}\n"
        label = row['present']
        output = f"{output} Label: {label}\n\n"
    return output


In [137]:
def classify_text(id: str, comment_text: str, topic: str, argument: str, samples: str) -> dict:

    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "user", "content": f"""
            Analyze the given comment in relation to a specific argument about {topic}. You need to:
            Identify if the comment makes use of the given argument. If it does, assign the label 1. If it does not, assign the label 0.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "label": "the label for the use of the argument in the comment"
            }}
            
            Some examples:

            {samples}


            Analyze the following comment in relation to the given argument:
            """},
        {"role": "user", "content": comment_text},
        ],
        response_format=ArgumentClassification,
        temperature=0
    )
    
    return completion.choices[0].message.content

In [147]:
def process_dataframe_comments(df: pd.DataFrame, topic: str, samples: str, n: int):
    label_to_argument = topic_label_to_argument.get(topic, {}) 
    with jsonl.open(f'yru_{topic}_identification_GPT_{n}shot.jsonl', mode='w') as writer:
        for idx, row in tqdm(df.iterrows(), desc="Processing comments", unit="comment", total=len(df)):
            comment_id = row['uid'] 
            comment_text = row['text']  
            comment_label = row['label']  

            argument_text = label_to_argument.get(comment_label)
            try:
                classification = classify_text(
                    id=comment_id, 
                    comment_text=comment_text,  
                    topic=topic,
                    argument=argument_text,
                    samples=samples
                )
                classification = json.loads(classification)
                output_entry = {"id": comment_id, "label": classification["label"]}
                #print(output_entry)
                writer.write(output_entry)
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                error_entry = {"id": comment_id, "label": 0}
                writer.write(error_entry)
                continue
            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
                error_entry = {"id": comment_id, "label": 0}
                writer.write(error_entry)
            continue
        

## Abortion

In [ ]:
topic = 'abortion'

for n in [5]:

    df = pd.read_csv(f'../../clean_data/yru_{topic}_with_negatives_main.csv')
    
    samples = prep_fewshot_samples(f'../../clean_data/yru_{topic}_with_negatives_{n}shot.csv', topic)

    process_dataframe_comments(df, topic, samples, n)

## Marijuana

In [ ]:
topic = 'marijuana'

for n in [1, 5]:

    df = pd.read_csv(f'../../clean_data/yru_{topic}_with_negatives_main.csv')
    
    samples = prep_fewshot_samples(f'../../clean_data/yru_{topic}_with_negatives_{n}shot.csv', topic)

    process_dataframe_comments(df, topic, samples, n)

## Obama

In [ ]:
topic = 'obama'

for n in [1, 5]:

    df = pd.read_csv(f'../../clean_data/yru_{topic}_with_negatives_main.csv')
    
    samples = prep_fewshot_samples(f'../../clean_data/yru_{topic}_with_negatives_{n}shot.csv', topic)

    process_dataframe_comments(df, topic, samples, n)

## Gay Rights

In [ ]:
topic = 'gayRights'

for n in [1, 5]:

    df = pd.read_csv(f'../../clean_data/yru_{topic}_with_negatives_main.csv')
    
    samples = prep_fewshot_samples(f'../../clean_data/yru_{topic}_with_negatives_{n}shot.csv', topic)

    process_dataframe_comments(df, topic, samples, n)